# Issue 
1. 금융업계의 재무제표는 일반 기업들의 재무제표와 계정과목 명이 다르다. 또한 IFRS로 표준화 되어 있지 않다. 
2. 금융업(투자은행, 은행, 상해보험, 손해보험, 카드사)은 여러 업종으로 나눠지지만, 크게 투자은행, 은행, 카드사와 보험사(상해보험, 손해보험) 두가지 업종으로 구분할 수 있다. 


In [2]:
!pip install pandas --upgrade
!pip install numpy --upgrade


  Using cached pandas-1.4.2-cp39-cp39-win_amd64.whl (10.5 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.5
    Uninstalling pandas-1.2.5:
      Successfully uninstalled pandas-1.2.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dart-fss 0.3.16 requires pandas<1.3.0,>=1.1.5; python_version >= "3.7", but you have pandas 1.4.2 which is incompatible.


In [1]:
! pip install dart_fss

  Using cached pandas-1.2.5-cp39-cp39-win_amd64.whl (9.3 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.2
    Uninstalling pandas-1.4.2:
      Successfully uninstalled pandas-1.4.2


In [1]:
import pandas as pd
import dart_fss as dart

In [2]:
# 다운받은 전종목 기본정보 불러오기 
all_stocks = pd.read_excel('./data_5655_20220311.xlsx')

# 상장기업 주식 코드 중 보통주만 걸러내기 
listed_companies = all_stocks[all_stocks['주식종류']=='보통주']

# DART API KEY 설정
api_key='7db60698e63bd621a006025500fe0436ee65cb47'
dart.set_api_key(api_key=api_key)

# DART 에 공시된 회사 리스트 불러오기
corp_list = dart.corp.get_corp_list()

# 상장 기업들의 기업코드와 회사 이름
listed_companies_corp_code = []
listed_companies_name = []

for stock_code in listed_companies['단축코드'] : 
    #corp_list.find_by_corp_name( ) 함수의 결과 데이터는 [[종목 코드]종목 이름]의 형태
    #이를 벗겨내기 위해서 corp_list.find_by_corp_name( )[0]으로 코딩함 
    try : 
        listed_companies_corp_code.append(corp_list.find_by_stock_code(stock_code).corp_code)
        listed_companies_name.append(corp_list.find_by_stock_code(stock_code).corp_name)
        
    except : 
        # corp_list
        print(stock_code)
        
corp_dic = { name:value for name, value in zip(listed_companies_corp_code, listed_companies_name) }

C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Output()

Output()

Output()

041140
220630
225850
329560
052190
039670
225330
050320
950180
138250
225860
270020
240340
330990
153360
066110
053660


# Solution 
1. 금융, 은행, 상해보험, 카드 라는 이름이 들어간 회사들의 계정명을 추출한다. 
2. 이름에 '보험'이 들어간 회사들의 계정명을 추출한다. 

비교한다. 
- 가장 많은 계정과목명이 들어간 회사를 찾는다. 
- 또한 계정과목명의 variation들을 찾아낸다. 

In [5]:
# Open Dart Reader 라이브러리로 추출해 내지 못한 기업들을 불러온다. 
error_log = pd.read_csv('./한국기업 재무정보 수집 산출물/error log.csv', encoding = 'cp949')

In [6]:
error_log['기업명'].str.contains('보험').sum()

4

'금융'이 포함된 회사 19    
'생명'이 포함된 회사 8  
'화재'가 포함된 회사 5  
'보험'이 포함된 회사 4  
'투자'가 포함된 회사 11   
'증권'이 포함된 회사 21  
'은행'이 포함된 회사 3  
'카드'가 포함된 회사 1  
총 72개의 회사  

**금융기업들을 리스트업** 

In [7]:
은행 = error_log[error_log['기업명'].str.contains('은행')]
금융 = error_log[error_log['기업명'].str.contains('금융')]
증권 = error_log[error_log['기업명'].str.contains('증권')]
투자 = error_log[error_log['기업명'].str.contains('투자')]
생명 = error_log[error_log['기업명'].str.contains('생명')]
보험 = error_log[error_log['기업명'].str.contains('보험')]
화재 = error_log[error_log['기업명'].str.contains('화재')]
카드 = error_log[error_log['기업명'].str.contains('카드')]

In [8]:
은행_금융_카드 = pd.concat([은행,금융,카드])
증권_투자 = pd.concat([증권, 투자])
생명_화재_보험 = pd.concat([생명, 화재,보험])

숫자를 스트링으로 바꾸는 함수

In [9]:
def unit_editor(x) : 
    to_add = 8-len(str(x))
    edited = to_add*'0'+str(x)
    return edited 

In [10]:
은행_금융_카드['DART 기업코드'] = 은행_금융_카드['DART 기업코드'].apply(lambda x : unit_editor(x))
증권_투자['DART 기업코드'] = 증권_투자['DART 기업코드'].apply(lambda x : unit_editor(x))
생명_화재_보험['DART 기업코드'] = 생명_화재_보험['DART 기업코드'].apply(lambda x : unit_editor(x))

## dart - fss 라이브러리 이용
dart-fss 라이브러리는 xbrl을 이용하는 것이 아닌, html에 직접 들어가서 정보를 찾아오는 방법이다.  
dart-fss 라이브러리의 extract 메서드를 활용하여 재무제표를 추출할 수 있다. 
이때 'separate' 옵션은 ~ 을 추출하는 것으로 default 값은 false이지만, 회사에 ~ 재무제표만 존재하는 경우 

In [11]:
# 상성화재손해보험, quarter, separate = False(default)
api_key='7db60698e63bd621a006025500fe0436ee65cb47'
dart.set_api_key(api_key=api_key)

'7db60698e63bd621a006025500fe0436ee65cb47'

In [12]:
bs_col_df = pd.DataFrame()
is_col_df = pd.DataFrame()
cf_col_df = pd.DataFrame()

for company in 은행_금융_카드['DART 기업코드'] : 
    
    try : 
        fs = dart.fs.extract(corp_code= company, bgn_de='20200101', report_tp = 'annual', fs_tp=('bs', 'is', 'cis', 'cf'), separate = True)
        bs_df = fs.show('bs', show_class = False, show_concept = True).T.reset_index()
        # level_1은 쓸모없는 설명 열 이므로 제거 
        bs_df.drop(['level_1'],axis=1, inplace=True)
        bs_cols = bs_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열
        # 새로운 bs_df
        bs_df=pd.DataFrame(list(bs_cols),columns=[corp_dic[company]])

        try : 
            # class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
            is_df = fs.show('cis', show_class = False, show_concept = True).T.reset_index()
            # level_1은 쓸모없는 설명 열 이므로 제거 
            is_df.drop(['level_1'],axis=1, inplace=True)
            is_cols = is_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열
            # 새로운 is_df
            is_df=pd.DataFrame(list(is_cols),columns=[corp_dic[company]])
        except : 
            # class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
            is_df = fs.show('is', show_class = False, show_concept = True).T.reset_index()
            # level_1은 쓸모없는 설명 열 이므로 제거 
            is_df.drop(['level_1'],axis=1, inplace=True)
            is_cols = is_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열
            # 새로운 is_df
            is_df=pd.DataFrame(list(is_cols),columns=[corp_dic[company]])

        # class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
        cf_df = fs.show('cf', show_class = False, show_concept = True).T.reset_index()
        # level_1은 쓸모없는 설명 열 이므로 제거 
        cf_df.drop(['level_1'],axis=1, inplace=True)
        # 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
        cf_cols = cf_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
        # 새로운 cf_df
        cf_df=pd.DataFrame(list(cf_cols),columns=[corp_dic[company]])

        bs_col_df = pd.concat([bs_col_df, bs_df],axis=1)
        is_col_df = pd.concat([is_col_df, is_df],axis=1)
        cf_col_df = pd.concat([cf_col_df, cf_df],axis=1)
        
    except : 
        pass
    
    

bs_col_df.to_excel('은행_금융_카드사 BS.xlsx')
is_col_df.to_excel('은행_금융_카드사 IS.xlsx')
cf_col_df.to_excel('은행_금융_카드사 CF.xlsx')

Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

...

  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\requests\adapters.py", line 501, in send
    raise ConnectionError(err, request=request)
requests.exceptions.ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py:1403: RuntimeWarning: Unable to extract financial statements: {'rcp_no': '20200330002949', 'corp_code': '00432102', 'corp_name': '한국금융지주', 'stock_code': '071050', 'corp_cls': 'Y', 'report_nm': '사업보고서 (2019.12)', 'flr_nm': '한국금융지주', 'rcept_dt': '20200330', 'rm': '연'}.
  warnings.warn(warnings_text, RuntimeWarning)


Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\urllib3\connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\http\client.py", line 288, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed

urllib3.exceptions.ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py", line 1387, in extract
    statements = analyze_report(report=report,
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py", line 1242, in analyze_report
    xbrl = report.xbrl
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\filings\reports.py", line 377, in xbrl
    self.load_xbrl()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\filings\reports.py", line 389, in load_xbrl
    xbrl_attached = self._get_xbrl()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\filings\reports.py", line 406, in _get_xbrl
    attached_files = self.find_all(**query)
  File "C:\Users\user

requests.exceptions.ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py:1403: RuntimeWarning: Unable to extract financial statements: {'rcp_no': '20200330002866', 'corp_code': '00126292', 'corp_name': '삼성카드', 'stock_code': '029780', 'corp_cls': 'Y', 'report_nm': '사업보고서 (2019.12)', 'flr_nm': '삼성카드', 'rcept_dt': '20200330', 'rm': '연'}.
  warnings.warn(warnings_text, RuntimeWarning)
